# ChiPy Mentorship 2 of 3

Since Post 1 my project work has varied between learning data science at a high level, expanding my Python foundation, and conducting exploratory data analysis. In this post I walk through the tools I’m using and my approach to an evolving goal line.

#### Learning and Project Clarity
Since my last post I have been working my way through the book, Data Science for Business (thanks to my mentor Aly for the purchase). The book provides a nice balance between high level discussion and gritty nuance. Most importantly, the text provides language for speaking to my interests and experiences in analytics, and defining my the stages and scope of my project.

See my previous post for information on my role as an analyst at an AmeriCorps non-profit and the data at my disposal.

The analysis I’m conducting involves profiling and causal modeling. I want to know which data points correlate most strongly to student outcomes and AmeriCorps Member performance. I am hoping I can then build a profile of successful (or unsuccessful) AmeriCorps Members (ACMs) and use this model to recommend metrics and goals that better align to student outcomes and ACM performance. Additionaly, this analysis could help advocate to school partners for the school settings in which our AmeriCorps Members best serve students’ individualistic needs.

#### Getting Data
The first task was to get my data into Python. The majority of our data are stored on Salesforce or Excel workbooks through SharePoint. Going into this project, I had already developed Python systems for getting and writing Excel documents to SharePoint. This was accomplished through a mapped network drive to the SharePoint server, allowing me to simply navigate the file structure as a local drive.

The Salesforce component was a bit more tricky, but offered an excellent opportunity to practice my Python fundamentals. The solution involed tuples and dictionaries, in addition to the list and pandas dataframe types I use most often.

I relied heavily on the package, `simple-salesforce`. This package allows me to query, create, and delete records, which has had a profound effect on my work pracitices. In a typical use of simple-salesforce, the user passes a traditional SOQL query into the Salesforce instance `cysh` and performs the `query_all` function.

Here is the set-up:

In [4]:
import pandas as pd
from simple_salesforce import Salesforce

with open('C:\\Users\\City_Year\\Desktop\\salesforce_credentials.txt', 'r') as f:
    read_data = f.read()
    sf_creds = eval(read_data)

# our Salesforce instance is referred to as cyschoolhouse (cysh)
cysh = Salesforce(instance_url=sf_creds['instance_url'],
                  password=sf_creds['password'],
                  username=sf_creds['username'],
                  security_token=sf_creds['security_token'])

And a simple query:

In [ ]:
querystring = (f"SELECT Id, Name FROM Assesment__c") # note the typo in Assesment__c, which took too long to discover
query_return = cysh.query_all(querystring)

In [ ]:
type(query_return)

In [ ]:
query_return.keys()

In [ ]:
len(query_return['records'])

In my case, I'm not working with datasets that are very large. In Chicago, we serve fewer than 3000 students, and we enlist fewer than 300 AmeriCorps Members. Therefore I can query the entire Salesforce object, `Assesment__c`, with whatever fields I'm interested in, and perform all the shaping and filtering with pandas.

I added this convenient function to iterate over the query response and shape it as a dataframe:

In [5]:
def get_cysh_df(sf_object, sf_fields, rename_id=False, rename_name=False, sf=cysh):
    sf_fields_str = ", ".join(sf_fields)
    querystring = (f"SELECT {sf_fields_str} FROM {sf_object}")
    query_return = cysh.query_all(querystring)

    query_list = []
    for row in query_return['records']:
        record = []
        for column in sf_fields:
            col_data = row[column]
            record.append(col_data)
        query_list.append(record)
    
    df = pd.DataFrame(query_list, columns=sf_fields)
    
    if rename_id==True:
        df.rename(columns={'Id':sf_object}, inplace=True)
    if rename_name==True:
        df.rename(columns={'Name':(sf_object+'_Name')}, inplace=True)

    return df

The use of this function is as follows:

In [6]:
assessment_df = get_cysh_df('Assesment__c', ['Id', 'Name'])
assessment_df.head()

,Id,Name
0,a041a00000E9Y3xAAF,a041a00000E9Y3x
1,a041a00000EmNs0AAF,a041a00000EmNs0
2,a041a00000EmOf8AAF,a041a00000EmOf8
3,a041a00000FV7AaAAL,a041a00000FV7Aa
4,a041a00000FVLdgAAH,a041a00000FVLdg


#### Cleaning Data
There are a lot of places I could start this project. I decided to start where I saw greatest opportunity: our observation and coaching data. I want to know if AmeriCorps Members who exercise best practices in tutoring are more impactful with students. This is data I had been intending to clean and aggregate, but which I put off because the 26 Excel workbooks created to track this data were not designed with aggregation in mind. The records are indexed by first names and nicknames, headers are inconsistent (and multi-indexed), and the file structure does not reflect standardized organization.

O&C data
- fuzzywuzzy to flag names to be manually fixed

#### Next Steps
Next I will determine student assessment progress by comparing prior year or start of year assessments to the recent winter assessment. Typically we set assessment goals fall to spring, so in this analysis I will need to calculate mid-year goals.